In [2]:
import os
import pandas as pd

# Chemin de la racine d'origine
source_dir = "Data"

# Chemin de la nouvelle racine pour les fichiers CSV
destination_dir = "Geolife Trajectories CSV/Data"

# Fonction pour créer le nouveau chemin tout en préservant la hiérarchie
def create_new_path(source_path, destination_root):
    relative_path = os.path.relpath(source_path, start=source_dir)
    new_path = os.path.join(destination_root, relative_path)
    return new_path

# Parcourir les fichiers dans la structure d'origine
for root, _, files in os.walk(source_dir):
    for file in files:
        if file.endswith(".plt"):
            plt_path = os.path.join(root, file)

            # Lire les données en sautant les 6 premières lignes
            with open(plt_path, "r") as plt_file:
                lines = plt_file.readlines()[6:]  # Ignorer les métadonnées
            
            # Découper les lignes en colonnes
            data = [line.strip().split(",") for line in lines]
            
            # Vérification des données valides (minimiser les erreurs d'entrée)
            data = [row for row in data if len(row) >= 7]
            
            # Création d'un DataFrame pandas
            columns = ['Latitude', 'Longitude', 'Unused', 'Altitude', 'Horodatage', 'Date', 'Heure']
            df = pd.DataFrame(data, columns=columns)
            
            # Supprimer la colonne inutile 'Unused'
            df.drop(columns=['Unused'], inplace=True)
            
            # Créer le chemin vers le répertoire de destination
            new_root = create_new_path(root, destination_dir)
            os.makedirs(new_root, exist_ok=True)  # Créer les dossiers si nécessaire
            
            # Chemin du fichier CSV
            csv_file = os.path.splitext(file)[0] + ".csv"
            csv_path = os.path.join(new_root, csv_file)
            
            # Sauvegarder le DataFrame en CSV
            df.to_csv(csv_path, index=False)
            
            print(f"Fichier converti : {plt_path} -> {csv_path}")


Fichier converti : Data\000\Trajectory\20081023025304.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081023025304.csv
Fichier converti : Data\000\Trajectory\20081024020959.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081024020959.csv
Fichier converti : Data\000\Trajectory\20081026134407.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081026134407.csv
Fichier converti : Data\000\Trajectory\20081027115449.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081027115449.csv
Fichier converti : Data\000\Trajectory\20081028003826.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081028003826.csv
Fichier converti : Data\000\Trajectory\20081029092138.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081029092138.csv
Fichier converti : Data\000\Trajectory\20081029093038.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081029093038.csv
Fichier converti : Data\000\Trajectory\20081103101336.plt -> Geolife Trajectories CSV/Data\000\Trajectory\20081103101336.csv


In [ ]:
import os
import pandas as pd

# Définir les chemins de dossier
input_base_path = "Data"  # Dossier source contenant les dossiers 000, 001, etc.
output_base_path = "Geolife Trajectories CSV_Merged/Data"  # Dossier de destination pour les fichiers fusionnés

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_base_path, exist_ok=True)

# Parcourir les dossiers dans le dossier source
for folder_name in os.listdir(input_base_path):
    folder_path = os.path.join(input_base_path, folder_name, "Trajectory")
    
    # Vérifier si c'est un dossier Trajectory
    if os.path.isdir(folder_path):
        # Initialiser une liste pour stocker les DataFrames des fichiers CSV
        csv_dataframes = []
        
        # Parcourir tous les fichiers CSV dans le dossier
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".csv"):
                file_path = os.path.join(folder_path, file_name)
                # Lire le fichier CSV et l'ajouter à la liste
                try:
                    df = pd.read_csv(file_path)
                    csv_dataframes.append(df)
                except Exception as e:
                    print(f"Erreur lors de la lecture de {file_path}: {e}")
        
        # Fusionner tous les DataFrames en un seul
        if csv_dataframes:
            merged_df = pd.concat(csv_dataframes, ignore_index=True)
            
            # Créer le chemin de destination
            output_folder_path = os.path.join(output_base_path, folder_name, "Trajectory")
            os.makedirs(output_folder_path, exist_ok=True)
            output_file_path = os.path.join(output_folder_path, f"user_{folder_name}.csv")
            
            # Sauvegarder le fichier fusionné
            merged_df.to_csv(output_file_path, index=False)
            print(f"Fichier fusionné enregistré : {output_file_path}")
        else:
            print(f"Aucun fichier CSV trouvé dans le dossier {folder_path}")
